In [919]:
import pandas as pd
import copy
from operator import itemgetter
from datetime import datetime, timedelta
import itertools
import math
import psycopg2

In [920]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except psycopg2.OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [921]:
connection = create_connection('postgres', 'postgres', '1234', 'localhost', '5432')

Connection to PostgreSQL DB successful


In [922]:
connection.autocommit = True
cursor = connection.cursor()


In [923]:
day = 7
# кол-во безномерных в месяц
unnum_number = 0

In [924]:
def date_convertation(entry):
    entry = str(entry).replace("-",".")
    date = entry.split()[0]
    day = date.split('.')[2]

    if day[0] == '0':
        day = day[1]
    month = date.split('.')[1]
    if month[0] == '0':
        month = month[1]

    year = date.split('.')[0]

    time = entry.split()[1]
    hour = time.split(':')[0]
    # if hour[0] == '0':
    #     hour = hour[1]
    minute = time.split(':')[1]
    if minute[0] == '0':
        minute = minute[1]
    if len(minute) > 2:
        minute = minute[:2]
    day, month, year, hour, minute = int(day) - 1, int(
        month), int(year), int(hour), int(minute)
    if month == 1:
        month = 0
    elif month == 2:
        month = 31
    if month == 3: 
        month = 59
    elif month == 4: 
        month = 90
    elif month == 5: 
        month = 120
    elif month == 6: 
        month = 151
    elif month == 7: 
        month = 181
    elif month == 8: 
        month = 212
    elif month == 9:
        month = 243
    elif month == 10:
        month = 273 
    elif month == 11:
        month = 304
    elif month == 12: 
        month = 334
    if year % 4 == 0 and month > 50:
        month += 1
    return round(year * 365 * 24 + month * 24 + day * 24 + hour + round(minute / 60, 2) - 2022 * 365 * 24, 2)

def reverse_convertation(entry):
    if entry == 0:
        return ''
    original_date = datetime(2022, 1, 1, 0, 0, 0)
    a = original_date + timedelta(hours=entry)
    a = str(a)
    b = a[8:10] + '.' + a[5:7] + '.' + a[:4] +  ' ' + a[11:16]
    return b
    

def moving(id1, id2):
    if date_convertation(id_startdate[id2]) <  date_convertation(id_enddate[id1]):
        dur2 = date_convertation(id_enddate[id2]) - date_convertation(id_startdate[id2])
        mov = date_convertation(id_enddate[id1]) - date_convertation(id_startdate[id2])
        print(dur2, mov, dur2+mov)
        id_startdate[id2] = id_enddate[id1]
        id_enddate[id2] = datetime(2022, 1, 1, 0, 0, 0) + timedelta(hours= date_convertation(id_startdate[id2]) + dur2)
        id_startidle[id2] =  datetime(2022, 1, 1, 0, 0, 0) + timedelta(hours= date_convertation(id_startidle[id2]) + mov)
        id_endidle[id2] = datetime(2022, 1, 1, 0, 0, 0) + timedelta(hours= date_convertation(id_endidle[id2]) + mov)
        if fixPeriod[id2] == 1:
            if id_startexp[id2] != -1:
                id_startexp[id2] = id_startdate[id2]
            if id_endexp[id2] != -1:
                id_endexp[id2] = id_enddate[id2]
        else:
            if id_startexp[id2] != -1 and id_endexp[id2] != -1:
                id_startexp[id2] = id_startdate[id2]
                id_endexp[id2] = id_enddate[id2]
            if id_endexp[id2] != -1:
                id_endexp[id2] = id_enddate[id2]
        # print(id_startdate[id2], '--', id_enddate[id2])
        return True
    else:
        return True


def timestamp_to_date(entry):

    entry = str(entry)
    if entry == '-1':
        return ''
    b = entry[8:10] + '.' + entry[5:7] + '.' + entry[:4] +  ' ' + entry[11:16]
    return b

In [925]:
def timestamp_to_string(array):
    for i in range(len(array)):
        if array[i] is not None:
            array[i] = str(array[i])
        if array[i] == 'NaT':
            array[i] = -1
    return array

In [926]:
#кушаем скважины
query = 'Select skv, ks_lk, coordinate_w, coordinate_l, deb_nef, name_ss from public.well_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()

events_df = pd.DataFrame(result, columns=["skv","ks_lk","coordinate_w","coordinate_l","deb_nef", "name_ss"])



# кол-во безномерных
unnum = 10
# Добыча на безномерных 
oil_unnum = 100

skv_id = [str(i) + "-" + str(list(events_df["ks_lk"])[list(events_df["skv"]).index(i)]) for i in list(events_df["skv"])]
skv = list(events_df["skv"]) 
coordinate_w = list(events_df["coordinate_w"])
skv_ks = list(events_df["ks_lk"])
coordinate_l = list(events_df["coordinate_l"])
deb_nef = list(events_df['deb_nef'])
skv_status = list(events_df["name_ss"])
status = list()
for i in skv_status:
    if 'работе' in i:
        status.append(1)
    else:
        status.append(0)
for i in range(len(deb_nef)):
    if deb_nef[i] is None:
        deb_nef[i] = 0
for i in range(1, unnum*2 + 1):
    coordinate_w.append("Unnum")
    coordinate_l.append("Unnum")
    status.append(1)
    if i <= unnum:
        skv.append("Положительная " + str(i))
        deb_nef.append(oil_unnum)
    else:
        skv.append("Отрицательная " + str(i))
        deb_nef.append(- 1* oil_unnum)
well_w = dict(zip(skv, coordinate_w))
well_l = dict(zip(skv, coordinate_l))
well_ks = dict(zip(skv, skv_ks))

In [927]:
query = 'Select brigade, brigade_n, brigade_type, well_id, status_brigade from public.brigade_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()



crew_df = pd.DataFrame(result, columns = ['brigade', 'brigade_n', 'brigade_type', 'well_id', 'status_brigade'])


brigade_id = list(crew_df['brigade'])

for i in range(len(brigade_id)):
    brigade_id[i] = str(brigade_id[i])
statusbrig = list(crew_df['status_brigade'])
brigade_status = dict(zip(brigade_id, statusbrig))
brigade_num = list(crew_df['brigade_n'])
brigade_type = list(crew_df['brigade_type'])

brigade_well = list(crew_df['well_id'])

brigid_well = dict(zip(brigade_id, brigade_well)) 


brig_unavailable = dict(zip(brigade_id, [0 for i in range(len(brigade_id))]))
match_id_type = dict(zip(brigade_id, brigade_type))
match_type_brig = {'Буровая бригада':1, 'ГНКТ': 2, 'ГРП': 3, 'КРС': 4}
cerera = 0
type_brig = list()
for i in range(len(brigade_type)):
    if cerera == 0:
        if 'бригада' in brigade_type[i] and 'Бригада' in brigade_id[i]:
            type_brig.append(6)
        elif 'бригада' in brigade_type[i]:
            type_brig.append(1)
    else:
        if 'бригада' in brigade_type[i] and 'Бригада' in brigade_id[i]:
            type_brig.append(1)
        elif 'бригада' in brigade_type[i]:
            type_brig.append(6)
    if brigade_type[i] == 'КРС':
        type_brig.append(4)
    elif brigade_type[i] == 'ГНКТ':
        type_brig.append(2)
    elif brigade_type[i] == 'ГРП':
        type_brig.append(3)

[6, 6, 6, 6, 6, 4, 4, 4, 4, 2, 4, 4, 2, 4, 1, 4, 4, 4, 4]


In [928]:
query = 'Select repair_id, repair, event_type, level, repair_type, brigade_id, repair_begin_date, repair_end_date, idlestart, idleend, repair_begin_exp, repair_end_exp, system_id, state, stop_req, qnstop, qnexp, well_id, well_event_id from public.plan_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()

plan_df = pd.DataFrame(result, columns = ['repair_id', 'repair', 'event_type', 'level', 'repair_type', 'brigade_id', 'repair_begin_date', 'repair_end_date', 'idlestart', 'idleend', 'repair_begin_exp', 'repair_end_exp', 'system_id', 'state', 'stop_req', 'qnstop', 'qnexp', 'well_id', 'well_event_id'])

query = 'Select repair_id, repair, event_type, level, repair_type, brigade_id, repair_begin_date, repair_end_date, idlestart, idleend, repair_begin_exp, repair_end_exp, system_id, state, stop_req, qnstop, qnexp, well_id, well_event_id, optim_id from public.fact_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()

fact_df = pd.DataFrame(result, columns = ['repair_id', 'repair', 'event_type', 'level', 'repair_type', 'brigade_id', 'repair_begin_date', 'repair_end_date', 'idlestart', 'idleend', 'repair_begin_exp', 'repair_end_exp', 'system_id', 'state', 'stop_req', 'qnstop', 'qnexp', 'well_id', 'well_event_id', 'optim_id'])

fact_df = fact_df[fact_df.optim_id == max(list(fact_df['optim_id']))]
    
repair_id = list(plan_df['repair_id']) + [i for i in list(fact_df['repair_id'])]
id = list(i + 1 for i in range(len(repair_id)))
event_type = list(plan_df['event_type']) + [i for i in list(fact_df['event_type'])]
repair = list(plan_df['repair']) + list(fact_df['repair'])
repair_type = list(plan_df['repair_type']) + list(fact_df['repair_type'])
brigade_id_plan = list(plan_df['brigade_id']) + list(fact_df['brigade_id'])
repair_begin_date = list(plan_df['repair_begin_date']) + list(fact_df['repair_begin_date'])
repair_begin_date = timestamp_to_string(repair_begin_date)

repair_end_date = list(plan_df['repair_end_date']) + list(fact_df['repair_end_date'])
repair_end_date = timestamp_to_string(repair_end_date)
idlestart = list(plan_df['idlestart']) + list(fact_df['idlestart'])
idlestart = timestamp_to_string(idlestart)
idleend = list(plan_df['idleend']) + list(fact_df['idleend'])
idleend = timestamp_to_string(idleend)
for i in range(len(idlestart)):
    if idlestart[i] == -1:
        idlestart[i] = repair_begin_date[i]
    if idleend[i] == -1:
        #print(i, repair_end_date[i])
        idleend[i] = repair_end_date[i]
repair_begin_exp = list(plan_df['repair_begin_exp']) + list(fact_df['repair_begin_exp'])
repair_begin_exp = timestamp_to_string(repair_begin_exp)
repair_end_exp = list(plan_df['repair_end_exp']) + list(fact_df['repair_end_exp'])
repair_end_exp = timestamp_to_string(repair_end_exp)
system_id = list(plan_df['system_id']) + list(fact_df['system_id'])
state = list(plan_df['state']) + list(fact_df['state'])
stop_req = list(plan_df['stop_req']) + list(fact_df['stop_req'])
qnstop = list(plan_df['qnstop']) + list(fact_df['qnstop'])
qnexp = list(plan_df['qnexp']) + list(fact_df['qnexp'])
well_id = list(plan_df['well_id']) + list(fact_df['well_id'])
well_event_id = list(plan_df['well_event_id']) + list(fact_df['well_event_id'])


# print(repair_begin_exp)    
#print(len(well_event_id))
level = list(plan_df['level']) + [i for i in list(fact_df['level'])]
for i in range(len(brigade_id_plan)):
    if brigade_id_plan[i] is None:
        brigade_id_plan[i] = -1
for i in range(len(well_event_id)):
    if well_event_id[i] is None or well_event_id[i] != well_event_id[i]:
        well_event_id[i] = -1
    else:
        well_event_id[i] = int(well_event_id[i])
for i in range(len(repair_begin_exp)):
    if repair_begin_exp[i] is None:
        repair_begin_exp[i] = -1
    if repair_end_exp[i] is None:
        repair_end_exp[i] = -1
oil_impact = list()
for i in range(len(qnstop)):
    if qnstop[i] is None:
        qnstop[i] = 0
    if qnexp[i] is None:
        qnexp[i] = 0
    if qnstop[i] != 0 and qnexp[i] == 0:
        qnexp[i] = qnstop[i]
    oil_impact.append(qnexp[i] - qnstop[i])

#print(oil_impact)



# print(id_repair[id1[0]])
# repair_id1 = []
# val = dict()
# a = len(id1)
# b = 0
# for i in range(a):
#     if repair_fact_id[i] not in repair_id1:
#         repair_id1.append(repair_fact_id[i])
#         val[repair_fact_id[i]] = optim_fact_id[i]
#     else:
#         if optim_fact_id[i] > val[repair_fact_id[i]]:
#             id1.pop(i - b)
#             b += 1
#             val[repair_fact_id[i]] = optim_fact_id[i]
# print(id1)




id_enddate = dict(zip(id, repair_end_date))
id_brig_fact = dict(zip(id, brigade_id_plan))
for i in id_brig_fact:
    if id_brig_fact[i] is not None:
        id_brig_fact[i] = str(int(id_brig_fact[i]))
id_fact = dict(zip(id, state))
id_well = dict(zip(id, well_id))
id_type_plan = dict(zip(id, repair_type))
id_repairid = dict(zip(id, repair_id))
id_wellevent_plan = dict(zip(id, [str(i) for i in well_event_id]))
id_repair = dict(zip(id, repair))
id_system = dict(zip(id, system_id))
id_startdate = dict(zip(id, repair_begin_date))
id_enddate = dict(zip(id, repair_end_date))
id_startidle = dict(zip(id, idlestart))
id_endidle = dict(zip(id, idleend))
id_startexp = dict(zip(id, repair_begin_exp))
id_endexp = dict(zip(id, repair_end_exp))
id_oilstop = dict(zip(id, stop_req))
id_oilimpact = dict(zip(id, oil_impact))
id_eventtype = dict(zip(id, event_type))
id_level = dict(zip(id, level))
for i in id_oilimpact:
    if id_fact[i] == 1:
        id_oilimpact[i] = 0
id_fixhours = dict()
id_doopt = dict()




# for i in id:
#     if id_fact[i] == 1:
#         continue
#     a = id_repairid[i]
#     for j in id:
#         if id_fact[j] == 1:
#             if id_repairid[j] == a and j != i:
#                 id_fact[i] = 1
#                 id_startdate[i] = id_startdate[j]
#                 id_enddate[i] = id_enddate[j]
#                 id_startidle[i] = id_startidle[j]
#                 id_endidle[i] = id_endidle[j]

# print(well_event_id[:554].count(71728))

3 2023-03-07 19:00:00
18 2023-03-31 00:00:00
40 2023-06-19 16:00:00
64 2023-05-26 01:00:00
75 2023-04-01 12:00:00
82 2023-06-08 00:00:00
84 2023-04-12 00:00:00
88 2023-04-29 12:00:00
89 2023-05-07 12:00:00
102 2023-03-31 00:00:00
120 2023-04-19 00:00:00
126 2023-03-31 00:00:00
128 2023-03-31 00:00:00
131 2023-03-31 12:00:00
140 2023-03-27 19:00:00
145 2023-03-31 00:00:00
155 2023-03-28 12:00:00
165 2023-03-31 00:00:00
166 2023-06-30 00:00:00
170 2023-06-06 00:00:00
174 2023-03-31 00:00:00
179 2023-04-14 12:00:00
211 2023-03-20 00:00:00
232 2023-03-18 00:00:00
234 2023-03-15 00:00:00
235 2023-03-29 00:00:00
238 2023-03-12 00:00:00
255 2023-03-25 19:30:00
259 2023-03-04 00:00:00
264 2023-03-31 19:30:00
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

In [932]:
def months(plus):
    month = datetime.now().month + plus
    if month > 12:
        year = '2024-'
    else:
        year = '2023-'
    if month == 12:
        return year + str(month)
    else:
        month = month % 12
    if month < 10:
        month = '0' + str(month)
        return year + str(month)
    else:
        return year + str(month)

In [934]:
if unnum_number != 0:

    if day > 19:
        cur_mon = 0
    else:
        cur_mon = 20 - day - (20 - day - day % 2) // 2
    if day < 8:
        las_mon = 0
    else:
        las_mon = day // 2 - 3
    for i in range(1, unnum_number*2 + cur_mon + las_mon + 1):
        id.append(len(id) + 1)
        id_repair[id[-1]] = '777777'
        id_repairid[id[-1]] = 'Безномерное ' + str(2 * i - 1)
        id_system[id[-1]] = 2
        id_fact[id[-1]] = 0
        id_eventtype[id[-1]] = 'Смена и ревизия ПО'
        id_level[id[-1]] = 'База'
        id_type_plan[id[-1]] = 'Смена УЭЦН'
        id_wellevent_plan[id[-1]] = i
        id_oilstop[id[-1]] = 1
        id_well[id[-1]] = 'Положительная ' + str(i % 10 if i % 10 != 0 else 10)
        id_brig_fact[id[-1]] = 0
        id_oilimpact[id[-1]] = 0
        if day < 10:
            ins_day = '0'+str(day)
        else:
            ins_day = str(day)
        if i <= cur_mon:
            if months(0)[-2:] == '02':
                id_endexp[id[-1]] = months(0)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(0)+'-29 00:00:00'
            id_startexp[id[-1]] = months(0)+'-'+ins_day + ' 00:00:00'
            id_startdate[id[-1]] = months(0)+'-21' + ' 00:00:00'
            id_enddate[id[-1]] = months(0) +'-25' + ' 00:00:00'
            id_startidle[id[-1]] = months(0)+'-21' + ' 00:00:00'
            id_endidle[id[-1]] = months(0) + '-25' + ' 00:00:00'
        elif i > cur_mon + unnum_number*2:
            if months(3)[-2:] == '02':
                id_endexp[id[-1]] = months(3)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(3)+'-29 00:00:00'
            id_startexp[id[-1]] = months(3)+'-01 00:00:00'
            id_startdate[id[-1]] = months(3)+'-01 00:00:00'
            id_enddate[id[-1]] = months(3)+'-05 00:00:00'
            id_startidle[id[-1]] = months(3)+'-01' + ' 00:00:00'
            id_endidle[id[-1]] = months(3) + '-05' + ' 00:00:00'
        elif i > cur_mon and i <= cur_mon + unnum_number:
            if months(1)[-2:] == '02':
                id_endexp[id[-1]] = months(1)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(1)+'-29 00:00:00'
            id_startexp[id[-1]] = months(1)+'-01 00:00:00'
            id_startdate[id[-1]] = months(1)+'-01 00:00:00'
            id_enddate[id[-1]] = months(1)+'-05 00:00:00'
            id_startidle[id[-1]] = months(1)+'-01' + ' 00:00:00'
            id_endidle[id[-1]] = months(1) + '-05' + ' 00:00:00'
        else:
            if months(2)[-2:] == '02':
                id_endexp[id[-1]] = months(2)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(2)+'-29 00:00:00'
            id_startexp[id[-1]] = months(2)+'-01 00:00:00'
            id_startdate[id[-1]] = months(2)+'-01 00:00:00'
            id_enddate[id[-1]] = months(2)+'-05 00:00:00'
            id_startidle[id[-1]] = months(2)+'-01' + ' 00:00:00'
            id_endidle[id[-1]] = months(2) + '-05' + ' 00:00:00'
        id.append(len(id) + 1)
        id_repair[id[-1]] = '777777'
        id_repairid[id[-1]] = 'Безномерное ' + str(2 * i)
        id_system[id[-1]] = 2
        id_fact[id[-1]] = 0
        id_type_plan[id[-1]] = 'Выход на режим'
        id_wellevent_plan[id[-1]] = i
        id_oilstop[id[-1]] = 1
        id_well[id[-1]] = 'Положительная ' + str(i % 10 if i % 10 != 0 else 10)
        id_brig_fact[id[-1]] = 0
        id_oilimpact[id[-1]] = 0
        if day < 10:
            ins_day = '0'+str(day)
        else:
            ins_day = str(day)
        if i <= cur_mon:
            id_startexp[id[-1]] = months(0)+'-'+ins_day + ' 00:00:00'
            if months(0)[-2:] == '02':
                id_endexp[id[-1]] = months(0)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(0)+'-29 00:00:00'
            id_startdate[id[-1]] = months(0)+'-25' + ' 00:00:00'
            id_enddate[id[-1]] = months(0) + '-29' + ' 00:00:00'
            id_startidle[id[-1]] = months(0)+'-25' + ' 00:00:00'
            id_endidle[id[-1]] = months(0) + '-29' + ' 00:00:00'
        elif i > cur_mon + unnum_number*2:
            id_startexp[id[-1]] = months(3)+'-01 00:00:00'
            if months(3)[-2:] == '02':
                id_endexp[id[-1]] = months(3)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(3)+'-29 00:00:00'
            id_startdate[id[-1]] = months(3)+'-05 00:00:00'
            id_enddate[id[-1]] = months(3)+'-09 00:00:00'
            id_startidle[id[-1]] = months(3)+'-05' + ' 00:00:00'
            id_endidle[id[-1]] = months(3) + '-09' + ' 00:00:00'
        elif i > cur_mon and i <= cur_mon + unnum_number:
            id_startexp[id[-1]] = months(1)+'-01 00:00:00'
            if months(1)[-2:] == '02':
                id_endexp[id[-1]] = months(1)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(1)+'-29 00:00:00'
            id_startdate[id[-1]] = months(1)+'-05 00:00:00'
            id_enddate[id[-1]] = months(1)+'-09 00:00:00'
            id_startidle[id[-1]] = months(1)+'-05' + ' 00:00:00'
            id_endidle[id[-1]] = months(1) + '-09' + ' 00:00:00'
        else:
            id_startexp[id[-1]] = months(2)+'-01 00:00:00'
            if months(2)[-2:] == '02':
                id_endexp[id[-1]] = months(2)+'-28 00:00:00'
            else:
                id_endexp[id[-1]] = months(2)+'-29 00:00:00'
            id_startdate[id[-1]] = months(2)+'-05 00:00:00'
            id_enddate[id[-1]] = months(2)+'-09 00:00:00'
            id_startidle[id[-1]] = months(2)+'-05' + ' 00:00:00'
            id_endidle[id[-1]] = months(2) + '-09' + ' 00:00:00'


In [936]:
# fix _ month
id_month = dict()
for i in id:
    if id_type_plan[i] == 'Перевод в ППД':
        id_month[i] = 1
    else:
        id_month[i] = 0

In [938]:
systems = {1: "System1", 2: "System2", 3: "System3"}

fail_type = {"A": 'stop_req не заполнена', "B": 'stop_req не равен 0 или 1', 'C': "Длительность простоя больше длительности мероприятия",
'D': "Часть простоя вне периода мероприятия", "E": "Скважина мероприятия отсутствует в списке скважин", 'F': 'Мероприятие Цереры не останавливает скважину',
'G': 'Колонка repair_begin_date не заполнена', 'H': 'Колонка repair_end_date не заполнена', 'I': 'Колонка idlestart не заполнена', 'J': 'Колонка idleend не заполнена', 'K': 'Мероприятие пересекается с предыдущим мероприятием в кортеже', 'L': 'repair_type не заполнена'}

In [939]:
# ошибки
fails = []
for i in id:
    # print(i, id_endidle[i], id_startidle[i], id_enddate[i], id_startdate[i])
    if id_oilstop[i] is None:
        fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'A'})
        id_oilstop[i] = 0
    elif id_oilstop[i] not in (0, 1) and id_system[i] != 1:
        fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'B'})
        id_oilstop[i] = 0
    elif id_system[i] == 1 and id_oilstop[i] != 2 and id_oilstop[i] != 1:
        fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'F'})
        id_oilstop[i] = 1
    try:
        date_convertation(id_startdate[i])
    except Exception:
         fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'G'})
    try:
        date_convertation(id_enddate[i])
    except Exception:
         fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'H'})
    try:
        date_convertation(id_startidle[i])
    except Exception:
         fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'I'})
    try:
        date_convertation(id_endidle[i])
    except Exception:
         fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'J'})
    if id_type_plan[i] is None:
        fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'L'})
    
        # print(i, id_repair[i])
    if date_convertation(id_endidle[i]) - date_convertation(id_startidle[i]) > date_convertation(id_enddate[i]) - date_convertation(id_startdate[i]):
        if id_fact[i] == 1:
            fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'C'})
            id_startidle[i] = id_startdate[i]
            id_endidle[i] = id_enddate[i]
        else:
            fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'C'})
    if date_convertation(id_startidle[i]) - date_convertation(id_startdate[i]) < 0 or date_convertation(id_enddate[i]) < date_convertation(id_endidle[i]):
        if id_fact[i] == 1:
            fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'D'})
            id_startidle[i] = id_startdate[i]
            id_endidle[i] = id_enddate[i]
        else:
            fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'D'})
    if id_well[i] not in skv:
        fails.append({"id": i, "id_repair": id_repair[i], "Критичность": "Критично", "Выгрузка из": systems[id_system[i]], "Ошибка": 'E'})
    
    

1 2023-04-03 12:00:00 2023-03-27 12:00:00 2023-04-03 12:00:00 2023-03-27 12:00:00
2 2023-04-14 12:00:00 2023-04-02 12:00:00 2023-04-14 12:00:00 2023-04-02 12:00:00
3 2023-04-02 00:00:00 2023-04-01 00:00:00 2023-04-02 00:00:00 2023-04-01 00:00:00
4 2023-03-07 19:00:00 2023-03-04 09:00:00 2023-03-07 19:00:00 2023-03-04 09:00:00
5 2023-06-25 05:00:00 2023-06-18 12:00:00 2023-06-25 05:00:00 2023-06-18 12:00:00
6 2023-03-31 00:00:00 2023-03-24 00:00:00 2023-03-31 00:00:00 2023-03-24 00:00:00
7 2023-03-31 00:00:00 2023-03-01 00:00:00 2023-03-31 00:00:00 2023-03-01 00:00:00
8 2023-05-08 12:00:00 2023-04-25 12:00:00 2023-05-08 12:00:00 2023-04-25 12:00:00
9 2023-03-30 09:21:19 2023-03-27 09:21:19 2023-03-30 09:21:19 2023-03-27 09:21:19
10 2023-04-22 20:00:00 2023-04-15 10:00:00 2023-04-22 20:00:00 2023-04-15 10:00:00
11 2023-04-06 12:00:00 2023-03-31 12:00:00 2023-04-06 12:00:00 2023-03-31 12:00:00
12 2023-04-16 12:00:00 2023-04-14 12:00:00 2023-04-16 12:00:00 2023-04-14 12:00:00
13 2023-04-18

In [940]:
for i in id_eventtype:
    if id_eventtype[i] == 'ВНС ГС':
        id_eventtype[i] = 'Ввод новых ГС'

In [941]:
# вычищение мероприятий из плана, которые есть в факте
id1 = id.copy()
for i in id:
    if id_fact[i] == 1:
        continue
    a = id_repair[i]
    b = id_well[i]
    for j in id:
        if id_fact[j] == 1:
            if id_repair[j] == a and j != i and id_well[j] == b:
                print(i, j)
                print(id_repair[j])
                id1.remove(i)
                del id_repair[i]
                del id_well[i]
id = id1
# print(len(id))

            

274


In [942]:
query = 'Select parentname, eventtypegroupname, repairkindname, repairtypename from public.repairkind_to_repairtype_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()



kindtorep_df = pd.DataFrame(result, columns = ['parentname', 'eventtypegroupname', 'repairkindname', 'repairtypename'])

parentname = list(kindtorep_df['parentname'])
groupname = list(kindtorep_df['eventtypegroupname'])
repkind = list(kindtorep_df['repairkindname'])
reptype = list(kindtorep_df['repairtypename'])
res = []
for i in range(len(parentname)):
    res.append(parentname[i] + groupname[i] + repkind[i])
eventtotype = dict(zip(res, reptype))



In [943]:
id_brigtype = dict()
for i in id:
    if id_system[i] == 1:
        id_brigtype[i] = 1
    if id_system[i] == 3:
        id_brigtype[i] = 5
    if id_system[i] == 2:
        if id_type_plan[i] == 'Выход на режим':
            id_brigtype[i] = 5
            continue
        a = id_level[i] + id_eventtype[i] + id_type_plan[i]
        if a in eventtotype.keys():
            a = eventtotype[a]
            if a == 'КРС':
                id_brigtype[i] = 4
            elif a == 'ГРП':
                id_brigtype[i] = 3
            else:
                id_brigtype[i] = 2
        elif id_fact[i] == 0:
            # print(i, a)
            id_brigtype[i] = 4
        else:
            id_brigtype[i] = 5


5 ГТМВвод новых ГССпуск ГНО
9 ГТМВвод новых ГССпуск ГНО
22 ГТМВвод новых ГССпуск ГНО
24 ГТМВвод из др. кат.Другие виды перевода
49 ГТМВвод новых ГССпуск ГНО
77 ГТМВвод новых ГССпуск ГНО
80 ГТМВвод новых ГССпуск ГНО
88 БазаПрочие КРС/ПРСОпределение состава, профиля притока, ТС, ИО
92 БазаПрочие КРС/ПРСОпредление профиля притока(комплекс приток-состав)
114 ГТМВвод новых ГССпуск ГНО
151 БазаПрочие КРС/ПРСОпределение профиля приемистости и ТС колонны
155 БазаВвод из БДПеревод в поглощающий
165 ГТМВвод новых ГССпуск ГНО
168 ГТМВвод новых ГССпуск ГНО
187 БазаПрочиеДругие виды перевода


In [945]:
for i in id:
    if i in id_fact:
        if id_fact[i] == 0 and (id_startdate[i] != id_startexp[i] or id_enddate[i] != id_endexp[i]):
            del id_brig_fact[i]

In [946]:
def poisk(arr, q):
    for i in arr:
        if q in i:
            return False
    return True 

In [947]:
def ind_eq(arr):
    res=[]
    
    for i in range(len(arr)- 1):
        intermediate = []
        for j in range(i+1,len(arr)):
            if arr[i] == arr[j] and arr[i] != 0 and poisk(res, i):
                if i not in intermediate:
                    intermediate.append(i)
                if poisk(res, j):
                    intermediate.append(j)
        if len(intermediate) != 0:
            res.append(intermediate)
    return res

In [949]:
def comb_creation(arr):
    n = len(arr)
    lens = [len(arr[i]) for i in range(n)]
    if n < 1:
        return []
    if n == 1:
        return arr
    if n == 2:
        combs = [[a, b] for a in range(lens[0]) for b in range(lens[1])]
    elif n == 3:
        combs = [[a, b, c] for a in range(lens[0]) for b in range(lens[1]) for c in range(lens[2])]
    elif n == 4:
        combs = [[a, b, c, d] for a in range(lens[0]) for b in range(lens[1]) for c in range(lens[2]) for d in range(lens[3])]
    elif n == 5:
        combs = [[a, b, c, d, e] for a in range(lens[0]) for b in range(lens[1]) for c in range(lens[2]) for d in range(lens[3]) for e in range(lens[4])]
    else:
        combs = [[a, b, c, d, e, f] for a in range(lens[0]) for b in range(lens[1]) for c in range(lens[2]) for d in range(lens[3]) for e in range(lens[4]) for f in range(lens[5])]
    for i in range(len(combs)):
        for j in range(len(combs[i])):
            combs[i][j] = arr[j][combs[i][j]]
    return combs

        
        

In [951]:
f = "D:\Учеба\Диплом\cortege_order.xlsx"


data = pd.read_excel(f, skiprows=3)
res = []
for column in data.columns:
    res.append(data[column].tolist())
res[0].pop(0)
res[0].pop(0)
index = []
for col in range(len(res)):
    for j in range(len(res[col])):
        if res[col][j] != res[col][j]:
            res[col][j] = 0
        else:
            try:
                res[col][j] = int(res[col][j])
            except:
                pass
for i in res[1:]:
    comb = []
    check = []
    for j in i[2:]:
        if j == 0:
            continue
        else:
            if i[2:].count(j) > 1 and j not in check:
                comb.append(i[2:].count(j))
                check.append(j)
    if len(comb) == 0:
        index.append(str(i[0]) + str(i[1]))
    else:
        col = 1
        for k in comb:
            col *= k
        if col > 10:
            for k in range(col):
                index.append(str(i[0]) + str(i[1]) + str(k // 10) + str (k % 10))
        else:
            for k in range(col):
                index.append(str(i[0]) + str(i[1]) + str (k % 10))
for i in range(1, len(res)):
    res[i] = res[i][2:]
# print(res)
# print(index)
corteges = dict(zip(index, [[] for _ in range(len(index))]))
curr_index = 0
i = 1
while i < len(res):
    # print(res[i], index[curr_index])
    combs = comb_creation(ind_eq(res[i]))
    # print(combs)
    if len(combs) == 1:
        for p in range(2):
            corteges[index[curr_index]] = ['' for _ in range(max(res[i]))]
            for j in range(len(res[i])):
                if res[i][j] != 0:
                    if res[i].count(res[i][j]) == 1:
                        corteges[index[curr_index]][res[i][j] - 1] = res[0][j]
            corteges[index[curr_index]][res[i][combs[0][p]] - 1] = res[0][combs[0][p]]
            curr_index += 1
        i += 1
        continue
    if len(combs) > 1:
        for k in combs:
            corteges[index[curr_index]] = ['' for _ in range(max(res[i]))]
            for j in range(len(res[i])):
                if res[i][j] != 0:
                    if res[i].count(res[i][j]) == 1:
                        corteges[index[curr_index]][res[i][j] - 1] = res[0][j]
            for j in k:
                corteges[index[curr_index]][res[i][j] - 1] = res[0][j]
            curr_index += 1
        i += 1
        continue
    corteges[index[curr_index]] = ['' for _ in range(max(res[i]))]
    for j in range(len(res[i])):
        if res[i][j] != 0:
            if res[i].count(res[i][j]) == 1:
                corteges[index[curr_index]][res[i][j] - 1] = res[0][j]
    # print(ind_eq(res[i]))
    curr_index += 1
    i += 1
# print(corteges)

In [952]:
def search(a, k):
    if len(a) == 0:
        return True
    for i in a:
        if k in i:
            return False
    return True

In [954]:
def dict_search(arr):
    for j in corteges:
        a = True
        cur_list = corteges[j].copy()
        if len(corteges[j]) == len(arr):
            for p in arr:
                if p not in cur_list:
                    a = False
                    break
                else:
                    cur_list.remove(p)
            if a:
                return j
        if a is False:
            continue
    return False

In [955]:
tup_events = []
for i in id:
    if id_system[i] == 1:
        well = id_well[i]
        for j in id:
            if id_well[j] == well and i != j and id_wellevent_plan[j] != '-1':
                id_wellevent_plan[i] = id_wellevent_plan[j]
                break
    if id_wellevent_plan[i] != '-1':
        tup_events.append((i, id_wellevent_plan[i], id_fact[i]))
#print(tup_events)
tup = []
for i in tup_events:
    curr_tup = []
    a = True
    for j in tup:
        if i in j:
            a = False
    if a:
        curr_tup.append(i)
    if len(curr_tup) == 0:
        continue
    for j in tup_events:
        if j[1] == curr_tup[0][1] and j not in curr_tup:
            curr_tup.append(j)
    if len(curr_tup) > 1:
        tup.append(curr_tup)
# print()
# print(tup)
# print(len(tup))
final = []
for i in tup:
    curr_types = []
    curr_nums = []
    for j in i:
        curr_types.append(id_type_plan[j[0]])
        curr_nums.append(j[0])
    
    res = dict_search(curr_types)
    #print(curr_types, curr_nums, res)
    if res is False:
        #print('Критическая ошибка', i)
        print('Критическая ошибка!')
        for j in i:
            #print(id_repair[j[0]], end =', ')
            print(id_type_plan[j[0]], end=', ')
        print()
        dates = []
    #if len(i) > 2:
        for j in i:
            dates.append([j[0], date_convertation(id_startdate[j[0]])])
        #print(dates)
        dates = sorted(dates, key = itemgetter(1))
        #print('SSSSSSSSSSSSSSSSSSSSSS' , dates)
        for i in range(len(dates)-1, -1, -1):
            if i == len(dates) - 1 and 'на режим' in id_type_plan[dates[i][0]]:
                break
            if 'на режим' in id_type_plan[dates[i][0]]:
                dates.append(dates.pop(i))
                break
        res = []
        for j in dates:
            res.append([j[0],id_wellevent_plan[j[0]]])

        final.append(res)
        continue
    else:
        res1 = curr_nums.copy()
        for p in range(len(curr_types)):
            res1[corteges[res].index(curr_types[p])] = curr_nums[p]
    res = []
    for j in res1:
        res.append([j, id_wellevent_plan[j]])
    final.append(res)

# for i in tup:
#     #print(i)
#     # for j in i:
#     #     print(id_repairid[j[0]], id_type_plan[j[0]])
#     # if len(i) == 3:
#     #     print('кортеж из 3 ивентов! -', i)
#     #     final.append([i[2], i[0], i[1]])
#     #     continue
#     dates = []
#     #if len(i) > 2:
#     for j in i:
#         dates.append([j[0], date_convertation(id_startdate[j[0]])])
#     #print(dates)
#     dates = sorted(dates, key = itemgetter(1))
#     print('SSSSSSSSSSSSSSSSSSSSSS' , dates)
#     for i in range(len(dates)-1, -1, -1):
#         if i == len(dates) - 1 and 'на режим' in id_type_plan[dates[i][0]]:
#             break
#         if 'на режим' in id_type_plan[dates[i][0]]:
#             dates.append(dates.pop(i))
#             break
#     res = []
#     for j in dates:
#         res.append([j[0],id_wellevent_plan[j[0]]])

#     final.append(res)
#     continue
print('-----------------------------------------------------------------------------')



tup = []
for i in id:
    if id_type_plan[i] == 'Перевод в ППД' and search(tup, i):
        well1 = id_well[i]
        curr_tup = []
        curr_tup.append([i, 0])
        for j in id:
            if search(curr_tup, j) and id_well[j] == well1:
                curr_tup.append([j, 0])
        if len(curr_tup) > 1:
            curr_tup = sorted(curr_tup, key = itemgetter(1))
            for j in range(len(curr_tup)):
                if id_type_plan[curr_tup[j][0]] == 'Перевод в ППД':
                    curr_tup.append(curr_tup.pop(j))
            for j in range(len(curr_tup)):
                if id_type_plan[curr_tup[j][0]] == 'Перевод скважины в ППД':
                    curr_tup.append(curr_tup.pop(j))
            tup.append(curr_tup)
print('-------', tup)
for i in tup:
    final.append(i)


for i in final:
    oil1 = id_oilimpact[i[0][0]]
    print(i, oil1)
    for j in range(1, len(i)):
        print(id_oilimpact[i[j][0]])
        if id_oilimpact[i[j][0]] == oil1:
            id_oilimpact[i[j - 1][0]] = 0
        oil1 = id_oilimpact[i[j][0]]
print()
print()
for i in final:
    oil1 = id_oilimpact[i[0][0]]
    for j in range(1, len(i)):
        print(oil1, id_oilimpact[i[j][0]])
    print('------')
print(final)

[[(1, '79102', 0), (178, '79102', 0)], [(2, '78604', 0), (12, '78604', 0), (112, '78604', 0)], [(5, '50966', 0), (50, '50966', 0)], [(6, '74958', 0), (47, '74958', 0)], [(8, '75650', 0), (136, '75650', 0)], [(9, '79210', 0), (34, '79210', 0), (97, '79210', 0)], [(13, '81804', 0), (32, '81804', 0)], [(14, '82811', 0), (88, '82811', 0)], [(16, '82593', 0), (268, '82593', 1)], [(17, '80396', 0), (22, '80396', 0), (108, '80396', 0)], [(29, '81794', 0), (128, '81794', 0)], [(33, '79202', 0), (220, '79202', 1), (251, '79202', 1)], [(43, '79212', 0), (49, '79212', 0), (76, '79212', 0)], [(45, '81066', 0), (234, '81066', 1)], [(54, '75680', 0), (223, '75680', 1)], [(56, '79095', 0), (145, '79095', 0)], [(57, '77974', 0), (109, '77974', 0)], [(62, '80518', 0), (263, '80518', 1)], [(64, '82828', 0), (92, '82828', 0), (116, '82828', 0)], [(66, '81366', 0), (164, '81366', 0)], [(68, '78111', 0), (157, '78111', 0)], [(72, '75692', 0), (118, '75692', 0)], [(73, '82806', 0), (151, '82806', 0)], [(77,

In [959]:
query = 'Select event_type1, event_type2, compatibility from public.simops_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()


simSheet_df = pd.DataFrame(result, columns = ['event_type1', 'event_type2', 'compatibility'])

type1 = list(simSheet_df['event_type1'])
type2 = list(simSheet_df['event_type2'])
compat1 = list(simSheet_df['compatibility'])
compat = list()
for i in compat1:
    if i is True:
        compat.append(1)
    else:
        compat.append(0)
# print(compat1[:20], compat[:20])
sim_dict = dict(zip(zip(type1, type2), compat))


[True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, False, True] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [961]:
#simops
simops = []

for i in range(len(id)):
    for j in range(i + 1, len(id)):
        if id_well[id[i]] == id_well[id[j]] and id_well[id[i]] != '0':
            if id_fact[id[i]] == 0 and id_fact[id[j]] == 0 and id_system[id[i]] != 1 and id_system[id[j]] != 1 and id_repair[id[i]] != '777777' and id_repair[id[j]] != '777777':
                if (id_eventtype[id[i]], id_eventtype[id[j]]) not in sim_dict.keys():
                    print('Error! No key in simops dictionary', i)
                    continue
                if sim_dict[(id_eventtype[id[i]], id_eventtype[id[j]])] == 1 and id_system[id[i]] + id_system[id[j]] == 5:
                    simops.append([id[i], id[j]])
# print(simops)
# for i in id:
#     if id_fact[i] == 0:
#         if id_system[i] == 3:
#             for j in id_well:
#                 if id_fact[j] == 0:
#                     if j != i and id_system[j] == 2 and id_well[j] == id_well[i]:
#                         simops.append([i, j])
# print(simops)
# for i in simops:
#     print(i)
#     print(id_repair[i[0]], id_repair[i[1]])



Error! No key in simops dictionary 90
Error! No key in simops dictionary 131
[]


In [962]:
# do optimization

fixPeriod = dict()
for i in id:
    if id_endexp[i] == id_enddate[i] and id_startexp[i] == id_startdate[i]:
        fixPeriod[i] = 1
        id_fixhours[i] = 1
    else:
        if id_system[i] == 1:
            fixPeriod[i] = 1
            id_fixhours[i] = 1
            continue
        if id_fact[i] == 1:
            fixPeriod[i] = 1
            id_fixhours[i] = 1
        else: 
            fixPeriod[i] = 0
for i in id:
    if id_system[i] == 3:
        if i in id_fixhours:
            continue
        if i in [j for i in simops for j in i]:
            id_fixhours[i] = 1
        elif id_oilstop[i] == 1:
            id_fixhours[i] = 1
        elif id_oilstop[i] == 0:
            fixPeriod[i] = 1
            id_fixhours[i] = 1
    else:
        id_fixhours[i] = 0
for i in id:
    if id_fact[i] == 1:
        id_doopt[i] = 1
    elif id_system[i] == 1:
        id_doopt[i] = 1
    elif id_system[i] == 3:
        if fixPeriod[i] == 1:
            if i in [j for i in simops for j in i]:
                id_doopt[i] = 1
            else:
                id_doopt[i] = 0
        else:
            id_doopt[i] = 0
            if i in [j for i in simops for j in i]:
                id_doopt[i] = 1
            elif id_oilstop[i] == 1:
                id_doopt[i] = 1
            
            
            # if id_doopt[i] == 0:
            #     if id_oilimpact[i] > 0:
            #         id_doopt[i] = 1
            #     elif id_oilimpact[i] == 0:
            #         id_doopt[i] = 0
            #     else:
            #         id_doopt[i] = 0

    elif id_system[i] == 2:
        id_doopt[i] = 1

# a = 0
# for i in id_doopt:
#     a += id_doopt[i]
# print(len(id_doopt))
# for i in id:
#     if i > 550:
#         print(i, fixPeriod[i])

In [963]:
for i in final:
    for j in range(1, len(i)):
        print(i[j], id_repair[i[j][0]], i[j-1], id_repair[i[j-1][0]])
        print(date_convertation(id_startdate[i[j][0]]), date_convertation(id_enddate[i[j - 1][0]]))
        if date_convertation(id_startdate[i[j][0]]) < date_convertation(id_enddate[i[j - 1][0]]) and id_fact[i[j][0]] == 0 and fixPeriod[i[j][0]] == 0:
            fails.append({"id": i[j][0], "id_repair": id_repair[i[j][0]], "Критичность": "Некритично", "Выгрузка из": systems[id_system[i[j][0]]], "Ошибка": 'K'})
            moving(i[j-1][0], i[j][0])
    print('-----')

[1, '79102'] 158554_6321 [178, '79102'] 157856_6321
10812.0 10812.0
-----
[12, '78604'] 164694_239 [112, '78604'] 157255_239
11244.0 10956.0
[2, '78604'] 164831_239 [12, '78604'] 164694_239
10956.0 11292.0
288.0 336.0 624.0
-----
[5, '50966'] 165283_4441 [50, '50966'] d093059c-7124-41a2-af9e-6ddc7b1dcd2d_4441
12804.0 12804.0
-----
[47, '74958'] 157271_46ПО [6, '74958'] 152950_46ПО
10896.0 10896.0
-----
[136, '75650'] 152590_318 [8, '75650'] 153483_318
11820.0 11820.0
-----
[9, '79210'] 158740_4679 [97, '79210'] 680c7325-3cb8-4872-9699-225c45c7928d_4679
10809.35 10809.35
[34, '79210'] 158741_4679 [9, '79210'] 158740_4679
10881.35 10881.35
-----
[13, '81804'] 165189_155 [32, '81804'] 164718_155
11244.0 11244.0
-----
[88, '82811'] 167628_1511 [14, '82811'] 167684_1511
11748.0 11748.0
-----
[16, '82593'] 167864_4644 [268, '82593'] 167154_4644
10671.25 10698.0
63.22999999999956 26.75 89.97999999999956
-----
[22, '80396'] 166987_4709 [17, '80396'] 109878e1-e95a-4b74-a4d7-784a622570fd_4709
11

In [964]:
for i in simops:
    for j in i:
        if id_doopt[j] == 0:
            print(j)
            simops.remove(i)
            break
simops1 = simops.copy()
for i in simops:
    for j in final:
        if i[0] in j and i[1] in j:
            simops1.remove(i)
simops = simops1.copy()
        


In [966]:
#fan event
id_fan = dict()
#print(id)
id_repair_cut = dict(zip(id_repair.keys(), [0 for i in range(len(id_repair.keys()))]))
for i in id_repair:
    if '_' in id_repair[i]:
        id_repair_cut[i] = id_repair[i][:id_repair[i].index('_')]
    else:
        id_repair_cut[i] = id_repair[i]
for i in id:
    if list(id_repair_cut.values()).count(id_repair_cut[i]) > 1 and id_repair[i] != '777777':
        id_fan[i] = 1
    else:
        id_fan[i] = 0
#print(id_fan)
# print(sum(id_fan.values()))
id_repair = id_repair_cut.copy()

30


In [968]:
# на какой скважине бригада закончила по факту и когда

for brig in brigade_id:
    curr_well = brigid_well[brig]
    a = 0
    if brig in id_brig_fact.values():
        for i in id_brig_fact:
            if id_brig_fact[i] == brig:
                b = date_convertation(id_enddate[i])
                if b > a:
                    a = b
                    curr_well = id_well[i]
                    brig_unavailable[brig] = a
    brigid_well[brig] = curr_well


In [970]:
for i in set(well_id):
    found = False
    for j in skv:
        if i == j:
            found = True
            continue
    if found == False: 
        print(i)

In [971]:
# импорт distanceoptim
query = 'Select cluster_1, cluster_2, distance from public.distance_optim'
eventSheet = cursor.execute(query)
result = cursor.fetchall()


distance_df = pd.DataFrame(result, columns = ['cluster_1', 'cluster_2', 'distance'])
cluster_1 = list(distance_df['cluster_1'])
cluster_2 = list(distance_df['cluster_2'])
for i in range(len(cluster_1)):
    cluster_1[i] = str(cluster_1[i])
    cluster_2[i] = str(cluster_2[i])
    if '.' in cluster_1[i]:
        cluster_1[i] = cluster_1[i][:-2]
    if '.' in cluster_2[i]:
        cluster_2[i] = cluster_2[i][:-2]
distance = list(distance_df['distance'])

clust_dist = dict(zip(zip(cluster_1, cluster_2), distance))


In [972]:
from haversine import haversine

In [973]:
def dist_def(i, w1, w2):
    # if i[0][0] == '119ПО':
    #     w1 = float(i[0][1])
    #     l1 = float(i[0][2])
    if w1[-1] == 'С':
        w1 = int(w1[:w1.index('°')]) + int(w1[w1.index('°')+1:w1.index("'")]) / 60 + int(w1[w1.index("'")+1: w1.index(",")])/3600 + float('0.' + w1[w1.index(',')+1: w1.index('"')])/3600
        l1 = i[0][2]
        l1 = int(l1[:l1.index('°')]) + int(l1[l1.index('°')+1:l1.index("'")]) / 60 + int(l1[l1.index("'")+1: l1.index(',')])/3600 + float('0.' + l1[l1.index(',')+1: l1.index('"')])/3600
    else:
        print(i)
        w1 = int(w1[:w1.index('°')]) + int(w1[w1.index('°')+2:w1.index("'")]) / 60 + float(w1[w1.index("'")+2: w1.index('"')])/3600 
        l1 = i[0][2]
        l1 = int(l1[:l1.index('°')]) + int(l1[l1.index('°')+2:l1.index("'")]) / 60 + float(l1[l1.index("'")+2: l1.index('"')])/3600
    w2 = i[1][1]
    # if i[1][0] == '119ПО':
    #     w2 = float(i[1][1])
    #     l2 = float(i[1][2])
    if w2[-1] == 'С':
        w2 = int(w2[:w2.index('°')]) + int(w2[w2.index('°')+1:w2.index("'")]) / 60 + int(w2[w2.index("'")+1: w2.index(",")])/3600 + float('0.' + w2[w2.index(',')+1: w2.index('"')])/3600
        l2 = i[1][2]
        l2 = int(l2[:l2.index('°')]) + int(l2[l2.index('°')+1:l2.index("'")]) / 60 + int(l2[l2.index("'")+1: l2.index(',')])/3600 + float('0.' + l2[l2.index(',')+1: l2.index('"')])/3600
    else:
        print(i)
        w2 = int(w2[:w2.index('°')]) + int(w2[w2.index('°')+2:w2.index("'")]) / 60 + float(w2[w2.index("'")+2: w2.index('"')])/3600 
        l2 = i[1][2]
        l2 = int(l2[:l2.index('°')]) + int(l2[l2.index('°')+2:l2.index("'")]) / 60 + float(l2[l2.index("'")+2: l2.index('"')])/3600
    # if i[0][0] == '509' and i[1][0] == '540':
    #     print(w1, l1, w2, l2)
    # delta_fi = abs(math.radians(l1) - math.radians(l2))
    # delta_lambda = abs(math.radians(w1) - math.radians(w2))
    dis = haversine((w1, l1), (w2, l2))
    #dis = 2*math.asin(min(math.sqrt(sqr(math.sin(delta_fi / 2)) + (1 - sqr(math.sin(delta_fi/2)) - sqr(math.sin((math.radians(l1) + math.radians(l2)) / 2)))*sqr(math.sin(delta_lambda / 2)) ), 1)) * 6371.008
    #dis = math.acos(round(math.sin(math.radians(l1)) * math.sin(math.radians(l2)) + math.cos(math.radians(l1)) * math.cos(math.radians(l2)) * math.cos(math.radians(w1) - math.radians(w2)), 11)) * 6371.008
    # if i[0][0] == '509' and i[1][0] == '540':
    #     print(dis)
    return dis

In [974]:
# расстояния 
wells = []

for i in skv:
    if i in well_id or "Положительная" in i or "Отрицательная" in i:
        wells.append([i, well_w[i], well_l[i]])
# print(wells)
matrix1 = list(itertools.product(wells, wells))
matrix = []
for i in matrix1:
    matrix.append(list(i))



for i in matrix:
    w1 = i[0][1]
    w2 = i[1][1]
    if w1 == "Unnum" or w2 == "Unnum":
        i.append(0)
        i.append(0)
        continue
    if well_ks[i[0][0]] == well_ks[i[1][0]]:
        dis = dist_def(i, w1, w2)
        i.append(round(dis*1000, 1))
        i.append(0)
        continue
    else:
        #print(i)
        #dis = dist_def(w1, w2)
        i.append(clust_dist[(well_ks[i[0][0]], well_ks[i[1][0]])] * 1000)
        i.append(round(clust_dist[(well_ks[i[0][0]], well_ks[i[1][0]])], 2))
# for i in matrix:
#     print(i)

[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['412', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['412', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['1442', '68° 31\' 28.685" N', '79° 51\' 24.728" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['1442', '68° 31\' 28.685" N', '79° 51\' 24.728" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['4441', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['4441', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 28.067" N', '79° 51\' 26.318" E'], ['4612', '68° 31\' 28.067" N', '79° 51\' 26.318" E']]
[['4443', '68° 31\' 2

In [979]:
for i in id_oilstop:
    if id_oilstop[i] is None:
        id_oilstop[i] = 0
    

In [981]:
id_brigid = dict()
for i in range(len(brigade_id)):
    id_brigid[brigade_id[i]] = i + 1
# print(id_brigid)

{'Бригада 5': 1, 'Бригада 8': 2, 'Бригада 1': 3, 'Бригада 2': 4, 'Бригада 3': 5, '16': 6, '58': 7, '60': 8, '61': 9, '65': 10, '120': 11, '142': 12, '161': 13, '484': 14, '1': 15, '62': 16, '57': 17, '152': 18, '439': 19}


In [983]:
tup_ev = []
for i in final:
    for j in i:
        tup_ev.append(j[0])
# print(tup_ev)
for i in id:
    if id_fact[i] == 1 and i not in tup_ev:
        id_doopt[i] = 0
        

[178, 1, 112, 12, 2, 50, 5, 6, 47, 8, 136, 97, 9, 34, 32, 13, 14, 88, 268, 16, 17, 22, 108, 128, 29, 220, 251, 33, 76, 49, 43, 234, 45, 223, 54, 56, 145, 109, 57, 263, 62, 116, 92, 64, 66, 164, 157, 68, 118, 72, 73, 151, 121, 77, 131, 78, 96, 142, 80, 87, 82, 86, 84, 137, 85, 89, 165, 90, 106, 168, 93, 169, 95, 271, 99, 100, 180, 245, 110, 211, 114, 189, 132, 147, 230, 217, 219, 224, 238, 261, 225, 229, 269, 232, 248, 237, 274, 249, 273, 264, 259, 198, 94, 240, 104, 244, 30]


In [984]:

start = True
for i in fails:
    if i['Критичность'] == 'Критично' and i['id'] not in [j[0] for i in final for j in i] and i['Ошибка'] in ('C', 'D') and id_fact[i['id']] == 1:
        i['Критичность'] = 'Некритично'
    if i['Критичность'] == 'Критично':
        start = False
        break


In [988]:
#экспорт скважин в ексель

if start:
    with pd.ExcelWriter("./Данные прототипа_18.БЕЗ_БН.xlsx") as excel:
        pd.DataFrame({"id Скважины":skv, "Координаты Ш":coordinate_w, "Координаты Д":coordinate_l, "Дебит нефти (ТР)":deb_nef, "Статус скважины": status}).to_excel(excel,"Скважины",index=False)
        # pd.DataFrame({"id":[repair_id.index(i) + 1 for i in repair_id], 
        # "id Мероприятия KS": repair_id, 
        # "id Мероприятия": repair, 
        # "id Скважины": well_id, 
        # "id Тип бригады": [5 if i != i else match_type_brig[match_id_type[i]] for i in brigade_id_plan], 
        # "datetime_start": repair_begin_date, 
        # "datetime_end": repair_end_date,
        # "длительность (ч)": [date_convertation(i) - date_convertation(repair_begin_date[repair_end_date.index(i)]) for i in repair_end_date], 
        # "длительность простоя (ч)": [date_convertation(i) - date_convertation(idlestart[idleend.index(i)]) for i in idleend], 
        # "Фиксированные периоды": fixPeriod,
        # "Остановка добычи" : stop_req,
        # "Влияние на добычу (в периодах после мероприятия, тн/сутки)": oil_impact,
        # "Бригада": ["" for i in brigade_id_plan],
        # "do_optimization": doOptimArray,
        # "needs_brigade": [1 if i == 5 else 0 for i in [5 if i != i else match_type_brig[match_id_type[i]] for i in brigade_id_plan]],
        # "fan_event": [1 if list(plan_df['repair']).count(i) > 1 else 0 for i in repair]}).to_excel(excel,"план",index=False)
        pd.DataFrame({"id":[i for i in range(1, len(brigade_id)+ 1)], "Имя Исполнителя":  brigade_id,"Номер бригады":brigade_num, "id Типа бригады":type_brig, "id Скважины (базирование)":[brigid_well[i] for i in brigid_well], "Статус": [brigade_status[i] for i in brigade_id], "Занята до": [reverse_convertation(brig_unavailable[i]) for i in brig_unavailable]}).to_excel(excel,"Бригады",index=False)
        pd.DataFrame({"id": id, "id Мероприятия KS": [id_repairid[i] for i in id], "id Мероприятия": [id_repair[i] for i in id], "id Скважины": [id_well[i] for i in id], 
        "Бригада": ["" if i not in id_brig_fact or id_brig_fact[i] == '-1' else id_brigid[id_brig_fact[i]] for i in id],
        "Тип": [1 if 'на режим' in id_type_plan[i] else 2 if 'Перевод в ППД' in id_type_plan[i] else 0 for i in id],
        "id Тип бригады": [id_brigtype[i] for i in id], "datetime_start": [timestamp_to_date(id_startdate[i]) for i in id], "datetime_end": [timestamp_to_date(id_enddate[i]) for i in id], 
        "Начать не раньше": [timestamp_to_date(id_startexp[i]) for i in id], 'Закончить не позже': [timestamp_to_date(id_endexp[i]) for i in id],
        "длительность (ч)": [date_convertation(id_enddate[i]) - date_convertation(id_startdate[i]) for i in id], "длительность простоя (ч)": [date_convertation(id_endidle[i]) - date_convertation(id_startidle[i]) for i in id],
        "Старт простоя через (ч)": [date_convertation(id_startidle[i]) - date_convertation(id_startdate[i]) for i in id],
        "Фиксированные периоды": [fixPeriod[i] for i in id],  
        "Фиксированные часы": [id_fixhours[i] for i in id], 
        "Фиксированный месяц": [id_month[i] for i in id],
        "Остановка добычи": [1 if id_oilstop[i] == 2 else id_oilstop[i] for i in id],
        "Влияние на добычу (в периодах после мероприятия, тн/сутки)":[0 if id_oilimpact[i] < 0.006 or id_fact[i] == 1 else id_oilimpact[i] for i in id],
        "do_optimization": [id_doopt[i] for i in id], 
        "needs_brigade": [0 if id_brigtype[i] == 5 or id_fact[i] == 1 and (i not in id_brig_fact or id_brig_fact[i] is None) else 1 for i in id],
        "fan_event": [id_fan[i] for i in id],
        "system_id": [id_system[i] for i in id], 'is_fact': [id_fact[i] for i in id]}).to_excel(excel,"Мероприятия",index=False)
        pd.DataFrame({'event_id1': [i[0] for i in simops], 'event_id2': [i[1] for i in simops]}).to_excel(excel,"SimOps",index=False)
        pd.DataFrame({"кортеж_id": [final.index(i) + 1  for i in final for j in range(len(i))], "кортеж_вариант": [1 for i in final for j in range(len(i))], "порядок": [j.index(i) + 1 for j in final for i in j],
        "id": [j[0] for i in final for j in i], "id Мероприятия KS": [id_repairid[i[0]] for j in final for i in j], "id Мероприятия":[id_repair[i[0]] for j in final for i in j], "id Скважины": [id_well[i[0]] for j in final for i in j],
        "Бригада": ["" if i[0] not in id_brig_fact or id_brig_fact[i[0]] == '-1' else id_brigid[id_brig_fact[i[0]]] for j in final for i in j],
        "Тип": [1 if 'на режим' in id_type_plan[i[0]] else 2 if 'Перевод в ППД' in id_type_plan[i[0]] else 0 for j in final for i in j],
        "id Тип бригады": [id_brigtype[i[0]] for j in final for i in j], "datetime_start": [timestamp_to_date(id_startdate[i[0]]) for j in final for i in j], "datetime_end": [timestamp_to_date(id_enddate[i[0]]) for j in final for i in j],
        "Начать не раньше": [timestamp_to_date(id_startexp[i[0]]) for j in final for i in j], 'Закончить не позже': [timestamp_to_date(id_endexp[i[0]])  for j in final for i in j],
        "длительность (ч)": [date_convertation(id_enddate[i[0]]) - date_convertation(id_startdate[i[0]]) for j in final for i in j], "длительность простоя (ч)": [date_convertation(id_endidle[i[0]]) - date_convertation(id_startidle[i[0]]) for j in final for i in j],
        "Старт простоя через (ч)": [date_convertation(id_startidle[i[0]]) - date_convertation(id_startdate[i[0]]) for j in final for i in j],
        "Фиксированные периоды": [fixPeriod[i[0]] for j in final for i in j], 'Фиксированные часы':[id_fixhours[i[0]] for j in final for i in j],"Остановка добычи": [1 if id_oilstop[i[0]] == 2 else id_oilstop[i[0]] for j in final for i in j],
        "Влияние на добычу (в периодах после мероприятия, тн/сутки)": [0 if id_oilimpact[i[0]] < 0.006 else id_oilimpact[i[0]] for j in final for i in j], "system_id": [id_system[i[0]] for j in final for i in j],
        "do_optimization": [id_doopt[i[0]] for j in final for i in j], "needs_brigade": [0 if id_brigtype[i[0]] == 5 else 1 for j in final for i in j], "fan_event": [id_fan[i[0]] for j in final for i in j], 'is_fact': [id_fact[i[0]] for j in final for i in j]}).to_excel(excel,"Мероприятия_кортеж",index=False)
        pd.DataFrame({"well_id1": [i[0][0] for i in matrix], "well_id2": [i[1][0] for i in matrix], "dist (m)": [i[2] for i in matrix], 'dist(hr)': [0 for i in matrix]}).to_excel(excel,"Расстояния",index=False)
        pd.DataFrame({"id Мероприятия (из мастер-системы)": [i['id_repair'] for i in fails], 'Скважина': [id_well[i['id']] for i in fails], "Критичность": [i['Критичность'] for i in fails], 
        "Выгрузка из": [i['Выгрузка из'] for i in fails], "Тип ошибки": [i['Ошибка'] for i in fails], "Ошибка": [fail_type[i['Ошибка']] for i in fails]}).to_excel(excel, "Ошибки", index=False)
        pd.DataFrame({"Обозначение": [i for i in fail_type], "Ошибка": [fail_type[i] for i in fail_type]}).to_excel(excel, "Типы ошибок", index=False)
else:
    print('Есть критические ошибки!')
    with pd.ExcelWriter("./ОшибкиОптимизатор_18.БЕЗ_БН.xlsx") as excel:
        pd.DataFrame({"id Мероприятия (из мастер-системы)": [i['id_repair'] for i in fails], 'Скважина': [id_well[i['id']] for i in fails], "Критичность": [i['Критичность'] for i in fails], 
        "Выгрузка из": [i['Выгрузка из'] for i in fails], "Тип ошибки": [i['Ошибка'] for i in fails], 'Ошибка': [fail_type[i['Ошибка']] for i in fails]}).to_excel(excel, "Ошибки", index=False)
        pd.DataFrame({"Обозначение": [i for i in fail_type], "Ошибка": [fail_type[i] for i in fail_type]}).to_excel(excel, "Типы ошибок", index=False)